In [ ]:
!pip3 install numpy==1.26.4
!pip3 install torch==2.3.1 torchvision torchaudio

In [ ]:
!pip3 install --upgrade jupyter
!pip3 install --upgrade ipywidgets

In [ ]:
!pip3 install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip3 install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

In [1]:
from unsloth import FastLanguageModel

max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

# https://github.com/unslothai/unsloth/issues/416
tokenizer.add_special_tokens({"pad_token": "<|reserved_special_token_0|>"})
model.config.pad_token_id = tokenizer.pad_token_id # updating model config
tokenizer.padding_side = 'right' # padding to right (otherwise SFTTrainer shows warning)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.0.
   \\   /|    GPU: NVIDIA GeForce RTX 2070. Max memory: 7.609 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


In [2]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [3]:
from datasets import load_dataset, Split

EOS_TOKEN = tokenizer.eos_token

def format_dataset(examples):
    texts = []
    for item in examples["prompt"]:
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = item + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }


train_dataset = load_dataset("andreirusu32/sql2ef", split=Split.TRAIN)
train_dataset = train_dataset.map(format_dataset, batched = True)

test_dataset = load_dataset("andreirusu32/sql2ef", split=Split.TEST)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported


trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

In [ ]:
trainer_stats = trainer.train()

In [ ]:
print(test_dataset[0])

In [4]:
import json
import os

class JsonFileManager:
    def __init__(self, filepath):
        self.filepath = filepath
        self.data = self._load_json()

    def does_item_exist(self, item_id):
        for item in self.data:
            if item['id'] == item_id:
                return True
        return False

    def _load_json(self):
        if os.path.exists(self.filepath):
            with open(self.filepath, 'r') as file:
                return json.load(file)
        return []

    def _save_json(self):
        with open(self.filepath, 'w') as file:
            json.dump(self.data, file, indent=4)



    def append_or_update(self, item):
        item_id = item.get('id')
        if item_id is None:
            raise ValueError("Item must have an 'id' field")

        for idx, existing_item in enumerate(self.data):
            if existing_item['id'] == item_id:
                self.data[idx] = item
                self._save_json()
                return

        self.data.append(item)
        self._save_json()

test_results_json_manager = JsonFileManager("test_results.json")

In [8]:
from tqdm import tqdm

FastLanguageModel.for_inference(model) # Enable native 2x faster inference

for item in tqdm(test_dataset):
    if test_results_json_manager.does_item_exist(item["query_id"]):
        print(f"Skipping query {item['query_id']}")
        continue

    prompt = item['prompt']

    inputs = tokenizer(
        [prompt],
        return_tensors = "pt",
    ).to("cuda")

    outputs = model.generate(
        **inputs,
        max_new_tokens = 256,
        use_cache = True,
    )

    decoded_output = tokenizer.batch_decode(outputs)[0]

    output_string = "**Output:**"
    starts_at = decoded_output.rfind(output_string) + len(output_string)

    completion = decoded_output[starts_at:].strip()


   # We could have looked for eos, but sometimes it just adds a description to the query, or other text,
    # (even though instructed not to) so we can look for ";".
    # eos_pos = completion.find(EOS_TOKEN)
    # if eos_pos >= 0:
    #     completion = completion[:eos_pos]

    eos_pos = completion.find(";")
    if eos_pos >= 0:
        completion = completion[:eos_pos + 1]
 

    test_results_json_manager.append_or_update({
        "id": item["query_id"],
        "linq": completion
    })


  0%|          | 0/792 [00:00<?, ?it/s]

Skipping query 31e0b6995b1944590a27586b0970cdea24dfe4419721c8a3a7d186175f8808bd
Skipping query eb6c91d414de92da6b85b94c1475aea48660a5435551095e959b559d49a1d80b
Skipping query 45dc41b0bd6268900ab3ec532febe5b2608a1b27003dd8ff25e6252456df6286
Skipping query 7b30b26548b3398edd8036a21b8e59ebf5a23a29eccede9ecac731b0b97de6f1
Skipping query 554d200951eac5fbd1f623eb524121e73a892018aa56bd626e63fcc099d68d74
Skipping query a74936f51df91b526fb493767ae63a7f90952011bd796e7a89e1ad28338bb078
Skipping query 2b4b1adb5d4890a17b13c505598499668cb8e6ca21751585a937083368eb8542
Skipping query aa92a1d72ecc5c0ab53cf2c97794ef2bfee73b97e890e2c6dc18d34c68973500
Skipping query eb7f3196c681b4af4e73ff9c6f8f097008da698921a3da5d0395aa1b74490471
Skipping query 5fc4fcb5f18cbd7f70ce7d712dadaa181a77e41cc14e716d1bb59e8c8ce14650
Skipping query 0e5ea380e0077d7b399d3ac6a15c81b859faf95ae91e259de8ffd0a52a0e63f6
Skipping query 9e243aae45d0a4817481afff943386ea7176775f9757c905ce0d17e5ecef83da
Skipping query bd84cb383c840a5021d079c7e

  4%|▍         | 30/792 [00:00<00:05, 136.64it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 24.00 MiB. GPU 